In [1]:
import glob
from bddl.activity import Conditions
import pathlib
import re

In [2]:
# Load the combined list
import json
scene_room_object_list = {}
with open(r"D:\ig_pipeline\artifacts\pipeline\combined_room_object_list.json", "r") as f:
    scene_room_object_list = json.load(f)["scenes"]

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\ig_pipeline\\artifacts\\pipeline\\combined_room_object_list.json'

In [3]:
import json, csv
from nltk.corpus import wordnet as wn
import networkx as nx
legit_synsets = {x.name() for x in wn.all_synsets()}

# Build the legit-synset graph
G = nx.DiGraph()
G.add_nodes_from(x.name() for x in wn.all_synsets())
for parent in wn.all_synsets():
    for child in parent.hyponyms():
        G.add_edge(parent.name(), child.name())
        
# Add the illegit-synset graph
with open(r"D:\bddl-dev\utils\custom_synsets.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        child = row["custom_synset"].strip()
        parent = wn.synset(row["hypernyms"].strip()).name()
        assert parent in G.nodes, "Could not find " + parent
        G.add_edge(parent, child)

In [4]:
# Get the activity filename map
act_to_fn = {}

from_b1k = glob.glob(r"D:\ObjectPropertyAnnotation\init_goal_cond_annotations\problem_files_verified_b1k\*")
from_b100 = glob.glob(r"D:\bddl-dev\bddl\activity_definitions\*")
for x in from_b1k + from_b100:
    p = pathlib.Path(x)
    if not p.is_dir():
        continue
    act_name = p.name.replace(" ", "_").replace("-", "_").replace("'", "_")
    if act_name in act_to_fn:
        print(f"Found duplicate for {act_name}")
    act_to_fn[act_name] = p

In [5]:
starting_scenes = {"house_single_floor"}

In [5]:
# The BEHAVIOR 200 activity list
activities_list = """attach a camera to a tripod
boil water
chop an onion
clean up broken glass
cleaning bathtub
fill a bucket in a small sink
folding piece of cloth
freeze pies
hanging up bedsheets
make a steak
make a strawberry slushie
melt white chocolate
mixing drinks
mowing the lawn
putting away Halloween decorations
putting away toys
putting up shelves
setting the fire
spraying for bugs
thawing frozen food""".replace(" ", "_").replace("-", "_").replace("'", "_").split("\n")
activities = set(activities_list)
if True:
    activities = set(act_to_fn.keys())
    activities_list = sorted(activities)

missing_acts = set(activities) - set(act_to_fn.keys())
print("Total acts", len(activities))
print("Missing acts", len(missing_acts))
assert not missing_acts, "Missing acts: " + str(missing_acts)

Total acts 1064
Missing acts 0


In [6]:
scene_mapping = {}
if False:
    with open(r"D:\ig_pipeline\metadata\b1k_activity_to_preselected_scenes.csv", newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            name = row["activity_name"].strip().replace(" ", "_").replace("-", "_").replace("'", "_")
            task_scenes = []
            for i in range(3):
                scene = row[f"scene{i}"].strip()
                if scene:
                    task_scenes.append(scene)
            assert name and task_scenes, f"Failed for {name}"
            scene_mapping[name] = task_scenes
    with open(r"D:\ig_pipeline\metadata\b100_activity_to_preselected_scenes.json") as f:
        mapping = json.load(f)
        for task, scenes in mapping.items():
            name = task.strip().replace(" ", "_").replace("-", "_").replace("'", "_")
            scene_mapping[name] = scenes
    
activities_found_in_mapping = set(scene_mapping.keys())
missing_mappings = activities - activities_found_in_mapping
# assert not missing_mappings, f"Missing mappings: {missing_mappings}"

In [12]:
import collections
from nltk.corpus import wordnet as wn

def counts_for(child, parent):  
    try:
        if nx.has_path(G, parent, child):
            return True
        else:
            c = wn.synset(child).name()
            p = wn.synset(parent).name()
            return nx.has_path(G, p, c)
    except Exception as e:
        # print(e)
        return False

def room_has_requirements(room_contents, room_reqs, verbose=False):
    success = True
    for req_name, req_number in room_reqs.items():
        found_stuff_for_req = 0
        for content_name, content_cnt in room_contents.items():
            if counts_for(content_name, req_name):
                found_stuff_for_req += content_cnt
                
        if found_stuff_for_req < req_number:
            if verbose:
                print(f"            Scene contains {found_stuff_for_req} {req_name} while {req_number} is required.")
            success = False
    return success

def any_room_has_requirements(s, required_rm, required_stuff, verbose=False):
    scene_contents = scene_room_object_list[s]
    matching_scene_rooms = [x for x in scene_contents.keys() if re.fullmatch("^" + required_rm + "_[0-9]+$", x)]
    for msr in matching_scene_rooms:
        if room_has_requirements(scene_contents[msr], required_stuff, verbose=verbose):
            return True
        else:
            if verbose:
                print(f"        {s}'s {msr} is not a match for {required_rm}")

    return False
            
def scene_compatible_with_task(s, task_requirements, verbose=False):
    requirements_by_rm = collections.defaultdict(collections.Counter)
    for obj, rm in task_requirements:
        assert not rm.endswith("_0")
        requirements_by_rm[rm.replace("_0", "")][obj] += 1
    
    for required_rm, required_stuff in requirements_by_rm.items():
        if not any_room_has_requirements(s, required_rm, required_stuff, verbose=verbose):
            if verbose:
                print(f"    Failed in {s} bc we couldn't find a suitable {required_rm}\n")
            return False
        
    return True

def evaluate_task(task_name, believe_assignments=False):
    task_file = act_to_fn[task_name] / "problem0.bddl"
    assert task_file.exists(), f"{task_name} file missing"
    with open(task_file, "r") as f:
        predefined_problem = "".join(f.readlines())
    # for m in re.findall("\(([A-Za-z_]+?) ", predefined_problem):
    #     if m not in pred_wl:
    #         return set()
    # init_states = predefined_problem.split(":init")[1].split(":goal")[0]
    # requirements = list(re.findall(r"\(inroom ([A-Za-z0-9_\.]+)_[0-9]+ ([A-Za-z0-9_\.]+)\)", init_states))
    dom = "omnigibson" if "ObjectPropertyAnnotation" in str(task_file) else "igibson"
    conds = Conditions(task_name, "potato", dom, predefined_problem=predefined_problem)
    objects = set(conds.parsed_objects.keys())
    raw_requirements = [cond[1:] for cond in conds.parsed_initial_conditions if cond[0] == "inroom"]
    assert all(len(req) == 2 for req in raw_requirements), task_name + ":" + str(raw_requirements)
    requirements = [(req.rsplit("_", 1)[0], rm) for req, rm in raw_requirements]
    good_scenes = {
        s for s in scene_room_object_list.keys() if s in starting_scenes and scene_compatible_with_task(s, requirements)
    }
    print("\n" + task_name)
    print("Reqs:", [x ])
    print("Objs:", objects)
    non_scene_objs = objects # - {obj for obj, rm in requirements}
    print("Nonscene objs:", non_scene_objs)
    if False:  # task_name in scene_mapping:
        assigned_scenes = set(scene_mapping[task_name]) & starting_scenes
        
        # Temporary forgiveness for things that do match something
        if good_scenes & starting_scenes or not assigned_scenes:
            return non_scene_objs, good_scenes
        
        missing_assigned_scenes = assigned_scenes - set(good_scenes)
        error_type = "Complete" if missing_assigned_scenes == assigned_scenes else "Partial"
        unexpected_matches = set(good_scenes) - assigned_scenes
        unexpected_match_str = f"Unexpected matches: {unexpected_matches}" if unexpected_matches and missing_assigned_scenes == assigned_scenes else ""
        if missing_assigned_scenes:
            print(f"\n{error_type} mismatch: {task_name} is assigned {assigned_scenes} but {missing_assigned_scenes} do not match init criteria. {unexpected_match_str}")
            for s in missing_assigned_scenes:
                scene_compatible_with_task(s, requirements, verbose=True)
        
        if believe_assignments:
            return non_scene_objs, good_scenes | assigned_scenes
    return non_scene_objs, good_scenes

In [13]:
improved_scene_mapping = {x: evaluate_task(x, believe_assignments=False) for x in activities}
unmatched_acts = [act for act, (_, scenes) in improved_scene_mapping.items() if not scenes]
print("\nUnmatched:", len(unmatched_acts))
print(unmatched_acts)


donating_clothing
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'shirt.n.01', 'floor.n.01', 'carton.n.02', 'dress.n.01', 'shoe.n.01', 'chest_of_drawers.n.01', 'boot.n.01', 'sweater.n.01', 'agent.n.01'}
Nonscene objs: {'shirt.n.01', 'floor.n.01', 'carton.n.02', 'dress.n.01', 'shoe.n.01', 'chest_of_drawers.n.01', 'boot.n.01', 'sweater.n.01', 'agent.n.01'}

shopping_at_warehouse_stores
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'checkout_counter.n.01', 'chicken.n.01', 'baguet.n.01', 'floor.n.01', 'corn_flake__container.n.01', 'shopping_cart.n.01', 'pomegranate.n.02', 'cream_cheese__container.n.01', 'agent.n.01', 'watermelon.n.02'}
Nonscene objs: {'checkout_counter.n.01', 'chicken.n.01', 'baguet.n.01', 'floor.n.01', 'corn_flake__container.n.01', 'shopping_cart.n.01', 'pomegranate.n.02', 'cream_cheese__container.n.01', 'agent.n.01', 'watermelon.n.02'}

setting_up_silent_auction
Reqs: ['D:\\bddl-dev\\bddl\\activity_def


cleaning_cups_in_living_room
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'floor.n.01', 'table.n.02', 'chalice.n.01', 'dust.n.01', 'rag.n.01', 'agent.n.01'}
Nonscene objs: {'floor.n.01', 'table.n.02', 'chalice.n.01', 'dust.n.01', 'rag.n.01', 'agent.n.01'}

organizing_boxes_in_garage
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'cabinet.n.01', 'ball.n.01', 'floor.n.01', 'carton.n.02', 'plate.n.04', 'shelf.n.01', 'saucepan.n.01', 'agent.n.01'}
Nonscene objs: {'cabinet.n.01', 'ball.n.01', 'floor.n.01', 'carton.n.02', 'plate.n.04', 'shelf.n.01', 'saucepan.n.01', 'agent.n.01'}

cook_quail
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'oven.n.01', 'countertop.n.01', 'electric_refrigerator.n.01', 'olive_oil__bottle.n.01', 'olive_oil.n.01', 'floor.n.01', 'quail.n.01', 'shelf.n.01', 'dutch_oven.n.01', 'agent.n.01'}
Nonscene objs: {'oven.n.01', 'countertop.n.01', 'electric_refrigerator.n.0


changing_bulbs
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'coffee_table.n.01', 'floor.n.01', 'ladder.n.01', 'broken__light_bulb.n.01', 'lamp.n.01', 'wall.n.01', 'light_bulb.n.01', 'agent.n.01'}
Nonscene objs: {'coffee_table.n.01', 'floor.n.01', 'ladder.n.01', 'broken__light_bulb.n.01', 'lamp.n.01', 'wall.n.01', 'light_bulb.n.01', 'agent.n.01'}

clean_conch_shells
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'countertop.n.01', 'water.n.06', 'floor.n.01', 'sink.n.01', 'dust.n.01', 'conch.n.01', 'rag.n.01', 'sand.n.01', 'agent.n.01'}
Nonscene objs: {'countertop.n.01', 'water.n.06', 'floor.n.01', 'sink.n.01', 'dust.n.01', 'conch.n.01', 'rag.n.01', 'sand.n.01', 'agent.n.01'}

cook_lamb
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'mint.n.04', 'grill.n.02', 'electric_refrigerator.n.01', 'cabinet.n.01', 'floor.n.01', 'wax_paper.n.01', 'clove.n.03', 'lamb.n.01', 'bowl.n.01', 'saltshak


spraying_fruit_trees
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'gate.n.01', 'atomizer.n.01', 'floor.n.01', 'fruit_tree.n.01', 'tree.n.01', 'pesticide.n.01', 'agent.n.01'}
Nonscene objs: {'gate.n.01', 'atomizer.n.01', 'floor.n.01', 'fruit_tree.n.01', 'tree.n.01', 'pesticide.n.01', 'agent.n.01'}

buy_natural_supplements
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'vitamin_pill__container.n.01', 'cash_register.n.01', 'floor.n.01', 'shelf.n.01', 'money.n.01', 'sack.n.01', 'checkout_counter.n.01', 'agent.n.01'}
Nonscene objs: {'vitamin_pill__container.n.01', 'cash_register.n.01', 'floor.n.01', 'shelf.n.01', 'money.n.01', 'sack.n.01', 'checkout_counter.n.01', 'agent.n.01'}

make_a_cheese_pastry
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'oven.n.01', 'saltshaker.n.01', 'countertop.n.01', 'sugar__sack.n.01', 'pastry.n.01', 'lemon_juice.n.01', 'cane_sugar.n.01', 'egg.n.02', 'mix.n.


collecting_aluminum_cans
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'bed.n.01', 'floor.n.01', 'pop.n.02', 'bucket.n.01', 'agent.n.01'}
Nonscene objs: {'bed.n.01', 'floor.n.01', 'pop.n.02', 'bucket.n.01', 'agent.n.01'}

organizing_items_for_yard_sale
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'coffee_table.n.01', 'floor.n.01', 'hanger.n.02', 'vase.n.01', 'rack.n.01', 'sweater.n.01', 'book.n.01', 'sofa.n.01', 'agent.n.01'}
Nonscene objs: {'coffee_table.n.01', 'floor.n.01', 'hanger.n.02', 'vase.n.01', 'rack.n.01', 'sweater.n.01', 'book.n.01', 'sofa.n.01', 'agent.n.01'}

putting_clean_laundry_away
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'floor.n.01', 'towel_rack.n.01', 'basket.n.01', 'hand_towel.n.01', 'bath_towel.n.01', 'agent.n.01'}
Nonscene objs: {'floor.n.01', 'towel_rack.n.01', 'basket.n.01', 'hand_towel.n.01', 'bath_towel.n.01', 'agent.n.01'}

dispose_of_fireworks
Req

...


freeze_cupcakes
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'countertop.n.01', 'cupcake.n.01', 'water.n.06', 'breakfast_table.n.01', 'package.n.02', 'cabinet.n.01', 'sink.n.01', 'floor.n.01', 'electric_refrigerator.n.01', 'cookie_sheet.n.01', 'agent.n.01', 'lid.n.02'}
Nonscene objs: {'countertop.n.01', 'cupcake.n.01', 'water.n.06', 'breakfast_table.n.01', 'package.n.02', 'cabinet.n.01', 'sink.n.01', 'floor.n.01', 'electric_refrigerator.n.01', 'cookie_sheet.n.01', 'agent.n.01', 'lid.n.02'}

make_citrus_punch
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'electric_refrigerator.n.01', 'orange_juice.n.01', 'countertop.n.01', 'pineapple_juice.n.01', 'bottle.n.01', 'ginger_beer.n.01', 'lemonade.n.01', 'pitcher.n.02', 'juice.n.01'}
Nonscene objs: {'electric_refrigerator.n.01', 'orange_juice.n.01', 'countertop.n.01', 'pineapple_juice.n.01', 'bottle.n.01', 'ginger_beer.n.01', 'lemonade.n.01', 'pitcher.n.02', 'juice.n.01'}



cleaning_driveway
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'floor.n.01', 'broom.n.01', 'driveway.n.01', 'grass.n.01', 'agent.n.01'}
Nonscene objs: {'floor.n.01', 'broom.n.01', 'driveway.n.01', 'grass.n.01', 'agent.n.01'}

hanging_address_numbers
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'floor.n.01', 'address.n.05', 'pedestal_table.n.01', 'wall.n.01', 'agent.n.01'}
Nonscene objs: {'floor.n.01', 'address.n.05', 'pedestal_table.n.01', 'wall.n.01', 'agent.n.01'}

make_a_cappuccino
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'espresso.n.01', 'water.n.06', 'cup.n.01', 'instant_coffee__jar.n.01', 'jar.n.01', 'countertop.n.01', 'cane_sugar.n.01', 'instant_coffee.n.01', 'bottle.n.01', 'cappuccino.n.01', 'foam.n.01', 'milk.n.01', 'blender.n.01', 'stirrer.n.02'}
Nonscene objs: {'espresso.n.01', 'water.n.06', 'cup.n.01', 'instant_coffee__jar.n.01', 'jar.n.01', 'countertop.n.01', 'c


cleaning_skis
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'ski.n.01', 'bathtub.n.01', 'floor.n.01', 'scraper.n.01', 'snow.n.01', 'agent.n.01'}
Nonscene objs: {'ski.n.01', 'bathtub.n.01', 'floor.n.01', 'scraper.n.01', 'snow.n.01', 'agent.n.01'}

cook_pasta
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'water.n.06', 'pasta.n.02', 'cabinet.n.01', 'floor.n.01', 'sink.n.01', 'stockpot.n.01', 'seawater.n.01', 'stove.n.01', 'bowl.n.01', 'saltshaker.n.01', 'salt.n.02', 'agent.n.01', 'lid.n.02'}
Nonscene objs: {'water.n.06', 'pasta.n.02', 'cabinet.n.01', 'floor.n.01', 'sink.n.01', 'stockpot.n.01', 'seawater.n.01', 'stove.n.01', 'bowl.n.01', 'saltshaker.n.01', 'salt.n.02', 'agent.n.01', 'lid.n.02'}

prepare_baking_pans
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'countertop.n.01', 'floor.n.01', 'vegetable_oil__bottle.n.01', 'pan.n.01', 'vegetable_oil.n.01', 'agent.n.01'}
Nonscene objs: {


make_gazpacho
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'black_pepper.n.02', 'water.n.06', 'countertop.n.01', 'gazpacho.n.01', 'lemon_juice.n.01', 'cucumber.n.02', 'tomato_juice.n.01', 'bottle.n.01', 'basil.n.03', 'bowl.n.01', 'salt.n.02', 'blender.n.0', 'container.n.01'}
Nonscene objs: {'black_pepper.n.02', 'water.n.06', 'countertop.n.01', 'gazpacho.n.01', 'lemon_juice.n.01', 'cucumber.n.02', 'tomato_juice.n.01', 'bottle.n.01', 'basil.n.03', 'bowl.n.01', 'salt.n.02', 'blender.n.0', 'container.n.01'}

cleaning_up_branches_and_twigs
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'grill.n.02', 'bin.n.01', 'floor.n.01', 'pruner.n.02', 'stick.n.01', 'branch.n.02', 'agent.n.01', 'yard.n.02'}
Nonscene objs: {'grill.n.02', 'bin.n.01', 'floor.n.01', 'pruner.n.02', 'stick.n.01', 'branch.n.02', 'agent.n.01', 'yard.n.02'}

clean_synthetic_hiking_gear
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']



clean_invisalign
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'water.n.06', 'floor.n.01', 'sink.n.01', 'sodium_carbonate.n.01', 'disinfectant__bottle.n.01', 'case.n.05', 'sodium_carbonate__jar.n.01', 'toothbrush.n.01', 'stool.n.01', 'disinfectant.n.01', 'retainer.n.03', 'rag.n.01', 'agent.n.01'}
Nonscene objs: {'water.n.06', 'floor.n.01', 'sink.n.01', 'sodium_carbonate.n.01', 'disinfectant__bottle.n.01', 'case.n.05', 'sodium_carbonate__jar.n.01', 'toothbrush.n.01', 'stool.n.01', 'disinfectant.n.01', 'retainer.n.03', 'rag.n.01', 'agent.n.01'}

sorting_books_on_shelf
Reqs: ['D:\\bddl-dev\\bddl\\activity_definitions\\washing_pots_and_pans']
Objs: {'coffee_table.n.01', 'floor.n.01', 'bookcase.n.01', 'comic_book.n.01', 'textbook.n.01', 'notebook.n.01', 'agent.n.01'}
Nonscene objs: {'coffee_table.n.01', 'floor.n.01', 'bookcase.n.01', 'comic_book.n.01', 'textbook.n.01', 'notebook.n.01', 'agent.n.01'}

freeze_meat
Reqs: ['D:\\bddl-dev\\bddl\\activity_defini

In [10]:
def get_tasks_possible_with_scenes(scene_set):
    b200_scene_mapping = {x: (improved_scene_mapping[x][0], set(improved_scene_mapping[x][1]) & scene_set) for x in activities if x in improved_scene_mapping}
    return {x: (objects, sorted(scenes)) for x, (objects, scenes) in b200_scene_mapping.items() if scenes}

In [11]:
print(len(get_tasks_possible_with_scenes({"house_single_floor"})))
print(len(get_tasks_possible_with_scenes({"house_single_floor", "Rs_int"})))
print(len(get_tasks_possible_with_scenes({"house_single_floor", "Wainscott_0_int"})))
print(len(get_tasks_possible_with_scenes({"house_single_floor", "Wainscott_0_int", "Rs_int"})))
print(len(get_tasks_possible_with_scenes({"Rs_int", "house_single_floor", "Wainscott_0_int", "Ihlen_0_int"})))
print(len(get_tasks_possible_with_scenes({"Rs_int", "Beechwood_0_int", "Wainscott_0_int", "Ihlen_0_int", "Merom_1_int", "house_single_floor"})))
print(len(get_tasks_possible_with_scenes({"Rs_int", "Beechwood_0_int", "Wainscott_0_int", "Ihlen_0_int", "Merom_1_int"})))

16
16
16
16
16
16
0


In [12]:
print("\n".join("1" if (improved_scene_mapping[x][1]) else "0" for x in activities_list))

1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
0
1
1
0


In [13]:
scene_mapping_repro = {x: scenes for x, (objects, scenes) in get_tasks_possible_with_scenes(starting_scenes).items()}
# print("\n".join(str(x) for x in sorted(scene_mapping_repro.items())))
flat_scene_mapping = [",".join(scene_mapping_repro[x] if x in scene_mapping_repro else []) for x in activities_list]
# print(len(flat_scene_mapping))
print("\n".join(flat_scene_mapping))

house_single_floor
house_single_floor
house_single_floor
house_single_floor
house_single_floor
house_single_floor
house_single_floor

house_single_floor
house_single_floor
house_single_floor
house_single_floor
house_single_floor
house_single_floor

house_single_floor

house_single_floor
house_single_floor



In [14]:
# preselection_fn = r"D:\ig_pipeline\metadata\b200_compatible_scenes.json"
# with open(preselection_fn, "w") as preselection_f:
#     json.dump(scene_mapping_repro, preselection_f, indent=4)
object_mapping = {x: sorted(improved_scene_mapping[x][0]) for x in activities_list}
objects_fn = r"D:\ig_pipeline\metadata\b200_objects.json"
with open(objects_fn, "w") as objects_f:
    json.dump(object_mapping, objects_f, indent=4)

In [ ]:
object_mapping["folding_piece_of_cloth"]